<a href="https://colab.research.google.com/github/Ishrak-DataScience/AIDrugDiscovery/blob/main/Ishraks_copy_of_NMG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from transformers import pipeline, set_seed
generator = pipeline('text-generation', model='gpt2')
set_seed(42)
generator("Hello, I'm a language model,", max_length=30, num_return_sequences=5)


Device set to use cuda:0
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=30) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


[{'generated_text': "Hello, I'm a language model, so I can write things that are easy to understand with a little bit of code. But when you think of words, it's hard to think of words that are as simple as a little word in a sentence.\n\nSo I'm going to use a little bit of code, and I'll talk a little bit about the syntax.\n\nThis is how you write your first line:\n\n$trans = new $trans [1, 2, 3, 4]; $trans -> write ( 'Hello, I'm a language model, so I can write things that are easier to understand with a little bit of code.' );\n\nThis code is pretty simple, but it really doesn't have to be. We want to write this code in a few lines, and we're going to use an expression, which is a shorthand for the literal of the language.\n\nIn this case, we're going to use a variable named trans that's a symbol. We want to write this expression as an expression, where we're going to look for a line that matches the line we want to write.\n\nThe syntax for writing a line like this is very simple:\n\

In [ ]:
!pip install rdkit


In [ ]:
!mamba install -c conda-forge rdkit -y


/bin/bash: line 1: mamba: command not found


In [ ]:
from rdkit import Chem
from rdkit.Chem import Descriptors

mol = Chem.MolFromSmiles("CCO")
print("Molecule OK:", mol is not None)
print("Molecular weight:", Descriptors.MolWt(mol))


Molecule OK: True
Molecular weight: 46.069


In [ ]:
!nvidia-smi


Sat Nov 15 14:27:44 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   53C    P0             27W /   70W |    1210MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer      ## 1st step of unfreezing

# Choose model name (small, fast version)
model_name = "gpt2"

# Download and cache both model and tokenizer
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

print("✅ GPT-2 model and tokenizer downloaded successfully!")
print("Model directory:", model_name)


✅ GPT-2 model and tokenizer downloaded successfully!
Model directory: gpt2


In [ ]:
import torch        ## 2nd step of unfreezing and mention layer no.

# ----------------------------
# ❄️ Freeze ALL layers
# ----------------------------
for param in model.parameters():
    param.requires_grad = False

# ----------------------------
# 🔓 Unfreeze ONLY one layer
# ----------------------------
layer_to_unfreeze = 11   # last layer, best choice

for param in model.transformer.h[layer_to_unfreeze].parameters():
    param.requires_grad = True

print(f"🔥 Unfrozen transformer layer: {layer_to_unfreeze}")

# ----------------------------
# 🔓 Also unfreeze the LM head
# ----------------------------
for param in model.lm_head.parameters():
    param.requires_grad = True

print("🔥 Unfrozen lm_head (output layer)")


🔥 Unfrozen transformer layer: 11
🔥 Unfrozen lm_head (output layer)


In [ ]:
!pip install -q torch transformers datasets rdkit pandas tqdm


In [ ]:
from google.colab import files
uploaded = files.upload()


Saving Components-smiles-cactvs.smi to Components-smiles-cactvs (1).smi


In [ ]:
import pandas as pd
from rdkit import Chem

smi_path = "Components-smiles-cactvs.smi"

# --- Read and extract only the SMILES strings ---
smiles = []
with open(smi_path, "r", encoding="utf-8", errors="ignore") as f:
    for line in f:
        line = line.strip()
        if not line or line.startswith("#"):
            continue
        first = line.split()[0]  # take only first column (SMILES)
        smiles.append(first)

print("Total lines read:", len(smiles))

# --- Validate and deduplicate using RDKit ---
uniq = []
seen = set()
for s in smiles:
    if s in seen:
        continue
    mol = Chem.MolFromSmiles(s)
    if mol is None:
        continue
    uniq.append(s)
    seen.add(s)

print(f"✅ Kept {len(uniq)} valid and unique SMILES out of {len(smiles)} total.")

# --- Save cleaned SMILES for training ---
pd.Series(uniq if len(uniq) > 0 else smiles).to_csv("smiles.txt", index=False, header=False)
print("📁 Saved cleaned file as smiles.txt (ready for fine-tuning)")


Total lines read: 48632


[14:28:53] SMILES Parse Error: syntax error while parsing: [Ru]|1|2|3|4|5|6|7|8(|[CH]9[C]|1=[C]|2[C]|3=[C]|49)|[C]%10=[C]|5[C]|6([C]|7=[C]|8%10)c%11cn(nn%11)c%12ccc(cc%12)[S](N)(=O)=O
[14:28:53] SMILES Parse Error: check for mistakes around position 5:
[14:28:53] [Ru]|1|2|3|4|5|6|7|8(|[CH]9[C]|1=[C]|2[C]
[14:28:53] ~~~~^
[14:28:53] SMILES Parse Error: Failed parsing SMILES '[Ru]|1|2|3|4|5|6|7|8(|[CH]9[C]|1=[C]|2[C]|3=[C]|49)|[C]%10=[C]|5[C]|6([C]|7=[C]|8%10)c%11cn(nn%11)c%12ccc(cc%12)[S](N)(=O)=O' for input: '[Ru]|1|2|3|4|5|6|7|8(|[CH]9[C]|1=[C]|2[C]|3=[C]|49)|[C]%10=[C]|5[C]|6([C]|7=[C]|8%10)c%11cn(nn%11)c%12ccc(cc%12)[S](N)(=O)=O'
[14:28:53] Explicit valence for atom # 23 Be, 4, is greater than permitted
[14:28:53] SMILES Parse Error: syntax error while parsing: O=C1NC(=O)c2cc3c([Ru]|4|5(|S6CCS|4CCS|5CC6)(N=C=S)|n7ccccc37)cc12
[14:28:53] SMILES Parse Error: check for mistakes around position 22:
[14:28:53] =C1NC(=O)c2cc3c([Ru]|4|5(|S6CCS|4CCS|5CC6
[14:28:53] ~~~~~~~~~~~~~~~~~~~~^
[14

✅ Kept 45757 valid and unique SMILES out of 48632 total.
📁 Saved cleaned file as smiles.txt (ready for fine-tuning)


In [ ]:
from datasets import load_dataset         ## 3rd step of unfreezing
from transformers import GPT2Tokenizer, DataCollatorForLanguageModeling

ds = load_dataset("text", data_files={"train": "smiles.txt"})

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token  # GPT-2 has no pad token

def tok_fn(batch):
    return tokenizer(batch["text"], truncation=True, max_length=128)

tok_ds = ds.map(tok_fn, batched=True, remove_columns=["text"])
collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
tok_ds


Map:   0%|          | 0/45757 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 45757
    })
})

In [ ]:
import torch  ## 4th step of unfreezing
from transformers import TrainingArguments, Trainer

# use the SAME 'model' that you already froze/unfroze
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

model.to(device)

# Training configuration
args = TrainingArguments(
    output_dir="./smiles_gpt2",
    overwrite_output_dir=True,
    num_train_epochs=1,          # later 3–5 if you want
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,  # effective batch ≈ 16
    learning_rate=5e-5,
    weight_decay=0.01,
    logging_steps=100,
    save_steps=500,
    save_total_limit=1,
    fp16=True,
    report_to="none",
)

# Trainer
trainer = Trainer(
    model=model,
    args=args,
    data_collator=collator,
    train_dataset=tok_ds["train"],
)

trainer.train()
trainer.save_model("./smiles_gpt2")
tokenizer.save_pretrained("./smiles_gpt2")

print("\n✅ Fine-tuning complete! Model saved to ./smiles_gpt2")


Using device: cuda


Step,Training Loss
100,2.836500
200,2.048100
300,1.870500
400,1.782400
500,1.715600
600,1.681000
700,1.655300
800,1.615000
900,1.607900
1000,1.574900



✅ Fine-tuning complete! Model saved to ./smiles_gpt2


In [ ]:
import torch
from transformers import GPT2LMHeadModel, TrainingArguments, Trainer

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

# Load base GPT-2
model = GPT2LMHeadModel.from_pretrained("gpt2")
model.resize_token_embeddings(len(tokenizer))
model.to(device)

# Training configuration
args = TrainingArguments(
    output_dir="./smiles_gpt2",
    overwrite_output_dir=True,
    num_train_epochs=1,                # ↑ later to 3–5 for higher quality
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,     # effective batch ≈ 16
    learning_rate=5e-5,
    weight_decay=0.01,
    logging_steps=100,
    save_steps=500,
    save_total_limit=1,
    fp16=True,
    report_to="none",
)

# Trainer
trainer = Trainer(
    model=model,
    args=args,
    data_collator=collator,
    train_dataset=tok_ds["train"],
)

trainer.train()
trainer.save_model("./smiles_gpt2")
tokenizer.save_pretrained("./smiles_gpt2")

print("\n✅ Fine-tuning complete! Model saved to ./smiles_gpt2")


Using device: cuda


Step,Training Loss
100,1.885800
200,1.358400
300,1.254200
400,1.204100
500,1.149000
600,1.130900
700,1.107300
800,1.072200
900,1.066500
1000,1.044000



✅ Fine-tuning complete! Model saved to ./smiles_gpt2


In [ ]:
from transformers import pipeline
from rdkit import Chem
import pandas as pd, time
import torch

# Load trained model
gen = pipeline(
    "text-generation",
    model="smiles_gpt2",     # <-- IMPORTANT: folder name only
    tokenizer="smiles_gpt2",
    device=0 if torch.cuda.is_available() else -1
)

seed = "CCO"  # or "" for random
outs = gen(
    seed,
    max_length=60,
    num_return_sequences=200,
    do_sample=True,
    top_k=50
)

valid = []
for o in outs:
    smi = o["generated_text"].strip().split()[0]
    if Chem.MolFromSmiles(smi):
        valid.append(smi)

valid = list(dict.fromkeys(valid))

print("Valid unique:", len(valid))

ts = time.strftime("%Y%m%d_%H%M%S")
filename = f"generated_valid_{ts}.csv"
pd.Series(valid, name="SMILES").to_csv(filename, index=False)

print("Saved file:", filename)


Device set to use cuda:0
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Both `max_new_tokens` (=256) and `max_length`(=60) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Valid unique: 2
Saved file: generated_valid_20251115_145046.csv


[14:50:46] SMILES Parse Error: extra close parentheses while parsing: CCOc1ccc(cc1)C(=O)N[CH](Cc2cccc(Cc3ccc(Cl)cc3)c2)C(C)(C)C)C(=O)NCCCC(O)=O)C(=O)N[CH](Cc4ccccc4)C(=O)NCCCC(O)=O)[CH](O)[CH](O)[CH](O)[CH]5CCCN5C(=O)N[CH](Cc6ccccc6)C(=O)N[CH](Cc13)c7cccc(Oc8cccc(Oc9ccc(Oc9c8)C(=O)N[CH](CCNC(N)=N)C(=O)NCCCCNC(O)=O)c9ccccc(Oc9n8)c7)c8Oc%10ccccc(Oc%10)c%11ccc(Oc%11)c%12ccccc(Oc%13ccc(Oc%12)c%13)c%14
[14:50:46] SMILES Parse Error: check for mistakes around position 58:
[14:50:46] c(Cl)cc3)c2)C(C)(C)C)C(=O)NCCCC(O)=O)C(=O
[14:50:46] ~~~~~~~~~~~~~~~~~~~~^
[14:50:46] SMILES Parse Error: Failed parsing SMILES 'CCOc1ccc(cc1)C(=O)N[CH](Cc2cccc(Cc3ccc(Cl)cc3)c2)C(C)(C)C)C(=O)NCCCC(O)=O)C(=O)N[CH](Cc4ccccc4)C(=O)NCCCC(O)=O)[CH](O)[CH](O)[CH](O)[CH]5CCCN5C(=O)N[CH](Cc6ccccc6)C(=O)N[CH](Cc13)c7cccc(Oc8cccc(Oc9ccc(Oc9c8)C(=O)N[CH](CCNC(N)=N)C(=O)NCCCCNC(O)=O)c9ccccc(Oc9n8)c7)c8Oc%10ccccc(Oc%10)c%11ccc(Oc%11)c%12ccccc(Oc%13ccc(Oc%12)c%13)c%14' for input: 'CCOc1ccc(cc1)C(=O)N[CH](Cc2cccc(Cc3ccc(Cl)cc3